<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_044_langchain_sequential_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Learning Langchain Chains**
Chains are the backbone of LangChain and allow you to sequence multiple steps for complex workflows. Start with simple chains and progress to more advanced ones.

- **Simple Chains**: Create a chain that connects a prompt to an LLM (similar to what you’ve already done).
- **Sequential Chains**: Link multiple prompts or LLM calls together for tasks like:
  - Summarizing a document and generating follow-up questions.
  - Translating text into multiple languages in one workflow.

In [ ]:
# !pip install langchain
# !pip install openai
# !pip install python-dotenv
# !pip install langchain-openai

In [ ]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnableSequence
from langchain.chains import SequentialChain, LLMChain

# Load environment variables from .env file
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
# Set the environment variable globally for libraries like LangChain
os.environ["OPENAI_API_KEY"] = api_key
# Print the API key to confirm it's loaded correctly
print("API Key loaded from .env:",os.environ["OPENAI_API_KEY"][0:30])

API Key loaded from .env: sk-proj-e1GUWruINPRnrozmiakkRM


## Simple Chains

### **Step-by-Step Explanation**

#### **1. Prompt Template**
- **What it is**: A template that defines the structure of the input you provide to the LLM.
- **Purpose**: It allows you to dynamically insert variables (e.g., `{question}`) into a predefined format.
- **Example**:
  ```python
  simple_prompt = PromptTemplate(input_variables=["question"], template="Answer the question: {question}")
  ```
  - `input_variables`: A list of placeholders (`["question"]`) that will be replaced with actual values.
  - `template`: The string structure for the prompt. In this case, the LLM will receive:
    ```
    Answer the question: [your question here]
    ```

---

#### **2. LLM Initialization**
- **What it is**: The `ChatOpenAI` object connects to OpenAI's LLMs (e.g., GPT-4) and handles the communication.

---

#### **3. Chaining with RunnableSequence**
- **What it is**: A modular way to connect components (like prompts and models) into a sequential workflow.
- **Purpose**: To send the output of one step (the prompt) as input to the next step (the LLM).
- **How it Works**:
  ```python
  simple_chain = simple_prompt | llm
  ```
  - `simple_prompt`: Creates the formatted text based on your input (e.g., `{"question": "What is the capital of France?"}` becomes `Answer the question: What is the capital of France?`).
  - `|`: Passes the output of the prompt directly to the LLM.
  - `llm`: Takes the formatted input and generates a response.

---

#### **4. Invoking the Chain**
- **What it is**: The `invoke` method runs the entire chain with the provided input.
- **Purpose**: Executes all the steps in the chain and returns the final output.
- **Example**:
  ```python
  response = simple_chain.invoke({"question": "What is the capital of France?"})
  ```
  - The `{"question": "What is the capital of France?"}` input replaces `{question}` in the prompt template.
  - The formatted prompt (`Answer the question: What is the capital of France?`) is sent to the LLM.
  - The LLM generates a response, e.g., `"The capital of France is Paris."`.

---

#### **5. Response Output**
- **What it is**: The final result generated by the LLM.

---

### **Why This is Important**
- **Modularity**: The chain is modular, meaning you can easily add or remove components (e.g., preprocessors, postprocessors).
- **Reusability**: The prompt and LLM setup can be reused across different workflows.
- **Simplicity**: By chaining steps together, you simplify the logic needed to handle complex tasks.

---

### **What You’re Learning Here**
1. **Prompt Design**: How to structure prompts dynamically using templates.
2. **LLM Querying**: How to interact with LLMs using LangChain’s `ChatOpenAI`.
3. **Chaining**: How to connect multiple components in a sequence using `RunnableSequence`.


## Example: Simple Chain

In [ ]:
# Step 1: Define a simple prompt template
simple_prompt = PromptTemplate(input_variables=["question"], template="Answer the question: {question}")

# Step 2: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4", temperature=0.5)

# Step 3: Create a runnable sequence
simple_chain = simple_prompt | llm  # This is the new RunnableSequence style

# Step 4: Run the chain
response = simple_chain.invoke({"question": "What is the capital of France?"})

# Step 5: Print the response
print("Simple Chain Response:", response.content)

Simple Chain Response: The capital of France is Paris.




### **When the LLM is Not the Last Step**

While it's common to place the **LLM** as the last step in the pipeline to generate the final output, there are scenarios where you might include additional steps after the model. It depends entirely on the workflow and the use case you're addressing.

#### **1. Post-Processing After the LLM**
- **Why?**: Sometimes, the raw output from the LLM needs to be cleaned, formatted, or transformed before it's useful.
- **Example**: Extracting structured information from a long LLM-generated response.
- **Code Example**:
  ```python
  from langchain_core.runnables import RunnableLambda

  # Define a post-processing function
  def extract_first_sentence(output: str) -> str:
      return output.split(".")[0] + "."

  post_processor = RunnableLambda(extract_first_sentence)

  # Add post-processing after the LLM
  pipeline = step_one | step_two | simple_prompt | llm | post_processor

  response = pipeline.invoke({"input_data": "Tell me a fun fact about space."})
  print("Processed Response:", response)
  ```

#### **2. Combining Results**
- **Why?**: If you're running multiple parallel LLM calls, you may need a final step to combine their outputs into one coherent result.
- **Example**: Translate text into multiple languages and then merge all translations into a single JSON.
- **Code Example**:
  ```python
  from langchain_core.runnables import RunnableLambda

  # Combine translations into a dictionary
  def combine_translations(outputs: dict) -> dict:
      return {"translations": outputs}

  combiner = RunnableLambda(combine_translations)

  # Parallel LLM chains
  pipeline = french_translation_chain | spanish_translation_chain | combiner
  ```

#### **3. Routing After the LLM**
- **Why?**: The LLM might provide intermediate insights, and the next step could depend on its output (e.g., decision-making logic).
- **Example**: The LLM categorizes a query into "math" or "language," and the output is routed to the appropriate post-processor.
- **Code Example**:
  ```python
  from langchain_core.runnables import RunnableLambda

  # Routing logic based on LLM output
  def route_based_on_output(output: str):
      if "math" in output.lower():
          return "math_pipeline"
      else:
          return "language_pipeline"

  router = RunnableLambda(route_based_on_output)

  # Add routing after LLM
  pipeline = step_one | llm | router
  ```

---

### **When the LLM is the Last Step**

Placing the LLM as the last step is more appropriate for straightforward workflows, such as:
1. **Text Completion**: When the LLM's response is directly consumed (e.g., generating a story or answering a question).
2. **Single-Step Outputs**: If there’s no need for further processing after the LLM.
3. **Conversational Agents**: Where the LLM produces the final response to display to a user.

**Example**:
```python
pipeline = step_one | step_two | llm
response = pipeline.invoke({"input_data": "Generate a creative story about AI."})
print("Final Output:", response)
```



## **Understanding Chains in LangChain**

#### **What Are Chains?**
Chains in LangChain are workflows that combine multiple steps, such as prompts, LLMs, tools, or custom functions, into a **sequential** or **modular process**. Each step in a chain takes input, processes it, and passes its output to the next step. This enables the creation of **complex, multi-step workflows** that involve large language models (LLMs) and external tools.

---

### **Why Use Chains?**

1. **Simplify Complex Tasks**:
   - Chains break down complex workflows into smaller, manageable steps.
   - Example: Summarizing a document → generating questions → translating the questions.

2. **Reusability**:
   - Individual steps (e.g., prompts or models) can be reused across multiple workflows.
   - Example: A summarization chain can be reused in workflows for question generation or translation.

3. **Modularity**:
   - Chains are modular, so you can easily add, remove, or replace steps in the process.
   - Example: Add a post-processing step after generating questions.

4. **Dynamic Input and Output**:
   - Chains allow you to connect inputs and outputs dynamically, making them adaptable to various tasks.
   - Example: Pass the output of a summarization step as input to a question-generation step.

5. **Debugging and Flexibility**:
   - Chains make it easier to debug and test intermediate outputs at each step of the workflow.

---

### **Types of Chains**

1. **Simple Chains**:
   - A single-step chain that connects a **prompt** to an **LLM**.
   - **Use Case**: Answering a question, text generation.
   - **Example**:
     ```python
     simple_chain = simple_prompt | llm
     response = simple_chain.invoke({"question": "What is the capital of France?"})
     ```

2. **Sequential Chains**:
   - Combine multiple steps in sequence, where the output of one step becomes the input to the next.
   - **Use Case**: Summarization → question generation → translation.
   - **Example**:
     ```python
     sequential_chain = SequentialChain(chains=[summary_chain, question_chain])
     response = sequential_chain.invoke({"input": "Your text here"})
     ```

3. **Parallel Chains**:
   - Run multiple tasks in parallel and combine their outputs.
   - **Use Case**: Translating text into multiple languages simultaneously.
   - **Example**:
     ```python
     parallel_chain = ParallelChain(chains={"French": french_chain, "Spanish": spanish_chain})
     response = parallel_chain.invoke({"input": "Your text here"})
     ```

4. **Router Chains**:
   - Dynamically route inputs to different sub-chains based on the input type or content.
   - **Use Case**: Handling queries that require specialized processing (e.g., math vs. language queries).
   - **Example**:
     ```python
     router_chain = MultiPromptChain(prompt_to_chain_map={"math": math_chain, "language": language_chain})
     response = router_chain.run({"question": "What is 5+5?"})
     ```

---

### **Components of a Chain**

1. **Prompt Templates**:
   - Define the structure of input for the LLM.
   - Example:
     ```python
     summary_prompt = PromptTemplate(input_variables=["text"], template="Summarize the following text: {text}")
     ```

2. **LLM (Language Model)**:
   - The LLM performs the core computation (e.g., generating responses or processing text).
   - Example:
     ```python
     llm = ChatOpenAI(model="gpt-4", temperature=0.7)
     ```

3. **Custom Functions**:
   - Use custom Python functions for pre- or post-processing.
   - Example:
     ```python
     process_output = RunnableLambda(lambda output: output.content.upper())
     ```

4. **Input and Output Keys**:
   - Define how data flows between steps in the chain.
   - Example: The first chain outputs `summary`, which becomes the input for the next chain.

---

### **Key Features of Chains**

1. **Dynamic Workflow**:
   - Chains can dynamically adapt to inputs, making them flexible for diverse use cases.

2. **Multi-Output Support**:
   - Chains can produce multiple outputs for downstream tasks (e.g., a summary and generated questions).

3. **Integration with Tools**:
   - Chains can interact with external tools, like calculators or APIs, in multi-step workflows.

4. **Error Handling**:
   - Chains simplify debugging by providing clear intermediate outputs at each step.

---

### **Use Cases of Chains**

1. **Text Processing Pipelines**:
   - Summarize text, extract key points, and generate questions.
   - Example: An educational app that provides summaries and quizzes.

2. **Conversational AI**:
   - Carry out multi-turn conversations with memory and contextual awareness.
   - Example: A chatbot that remembers user preferences.

3. **Data Transformation**:
   - Convert text into structured formats, such as JSON or CSV.
   - Example: Extracting data from documents for automation workflows.

4. **Document Search and QA**:
   - Search a knowledge base for relevant documents, summarize them, and answer questions.
   - Example: Customer support bots or legal document analysis.

5. **Multilingual Translation**:
   - Translate text into multiple languages in a single workflow.
   - Example: Cross-border customer communication tools.

---

### **Best Practices**

1. **Start Simple**:
   - Build small chains first (e.g., prompt → LLM) and expand as needed.

2. **Debug Intermediate Outputs**:
   - Test and print outputs at each step to ensure the workflow is functioning correctly.

3. **Use Modular Components**:
   - Design chains with reusable steps for flexibility and scalability.

4. **Specify Input/Output Keys**:
   - Explicitly define input and output keys to prevent data flow issues.

5. **Test with Real Data**:
   - Use realistic inputs to validate the performance and robustness of the chain.



## Example: Sequential Chain

In [ ]:
# Step 1: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini", temperature=0.5)

# Step 2: Define the first prompt (summarization)
summary_prompt = PromptTemplate(input_variables=["input"], template="Summarize the following text: {input}")
summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="summary")

# Step 3: Define the second prompt (question generation)
question_prompt = PromptTemplate(input_variables=["summary"], template="Generate 3 questions based on this summary: {summary}")
question_chain = LLMChain(prompt=question_prompt, llm=llm, output_key="questions")

# Step 4: Create a sequential chain
sequential_chain = SequentialChain(
    chains=[summary_chain, question_chain],
    input_variables=["input"],  # Input to the first chain
    output_variables=["summary", "questions"]  # Final output keys
)

# Step 5: Run the chain
text_to_summarize = "LangChain is a framework that simplifies building applications with large language models."
response = sequential_chain.invoke({"input": text_to_summarize})

# Step 6: Print the response
print("Summary:", response["summary"])
print("\n-----------  Questions   -----------\n")
print(response["questions"])


Summary: LangChain is a framework designed to make it easier to develop applications using large language models.

-----------  Questions   -----------

1. What are the primary features of the LangChain framework that facilitate the development of applications using large language models?  
2. How does LangChain simplify the integration of large language models into various application types?  
3. In what ways can developers benefit from using LangChain when working with large language models compared to traditional methods?  


### Key Code Elements


---

### **1. Defining Prompts for Each Step**

```python
summary_prompt = PromptTemplate(input_variables=["input"], template="Summarize the following text: {input}")
question_prompt = PromptTemplate(input_variables=["summary"], template="Generate 3 questions based on this summary: {summary}")
```

- **Explanation**:
  - The first prompt (`summary_prompt`) takes an input text (`input`) and asks the LLM to summarize it.
  - The second prompt (`question_prompt`) uses the output from the first step (`summary`) as its input to generate questions.
- **Key Concept**: Prompts in sequential chains must match the input/output flow between steps.

---

### **2. Creating Individual Chains**

```python
summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="summary")
question_chain = LLMChain(prompt=question_prompt, llm=llm, output_key="questions")
```

- **Explanation**:
  - **`summary_chain`**: Connects the `summary_prompt` to the LLM and outputs the result as `summary`.
  - **`question_chain`**: Connects the `question_prompt` to the LLM and outputs the result as `questions`.
- **Key Concept**: Use `output_key` to define what each chain outputs so that subsequent chains can use it.

---

### **3. Creating the Sequential Chain**

```python
sequential_chain = SequentialChain(
    chains=[summary_chain, question_chain],
    input_variables=["input"],  # Input to the first chain
    output_variables=["summary", "questions"]  # Final output keys
)
```

- **Explanation**:
  - **`chains`**: A list of chains to run in sequence. Each chain's output is passed as the next chain's input.
  - **`input_variables`**: Specifies the inputs for the first chain (`input` in this case).
  - **`output_variables`**: Specifies which outputs to return from the final chain(s).
- **Key Concept**: The sequential chain ensures data flows from one step to the next in the correct order.

---

### **4. Running the Chain**

```python
response = sequential_chain.invoke({"input": text_to_summarize})
```

- **Explanation**:
  - **Input**: You pass `{"input": text_to_summarize}` as the starting input to the chain.
  - **Output**: The chain returns a dictionary containing the outputs of all steps defined in `output_variables` (`summary` and `questions`).
- **Key Concept**: `invoke()` executes the entire sequence and gathers all specified outputs.

---

### **Overall Flow**

1. **Input (`"input"`)** → Passed into `summary_chain`.
2. **Output (`"summary"`)** → Generated by `summary_chain` and passed as input to `question_chain`.
3. **Output (`"questions"`)** → Generated by `question_chain` as the final result.





### **Why Use `LLMChain` and `SequentialChain` in This Example?**

The reason for using the structure with `LLMChain` and `SequentialChain` instead of `RunnableSequence` (`|` operator) comes down to **complexity management**, **reusability**, and **functionality differences**.

1. **Clear Input/Output Mapping**
   - **`LLMChain`** allows you to explicitly define:
     - **Input Keys**: The variable(s) required for the step (e.g., `"input"` for summarization).
     - **Output Keys**: The variable(s) produced by the step (e.g., `"summary"`).
   - In a multi-step chain like this (summarization → question generation), these explicit mappings are crucial to ensure that:
     - The output of `summary_chain` (`summary`) feeds correctly into the `question_chain` as its input.
   - This explicit input/output structure is harder to achieve with `RunnableSequence`.

   **Example**:
   ```python
   summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="summary")
   question_chain = LLMChain(prompt=question_prompt, llm=llm, output_key="questions")
   ```
   - Here, `summary_chain` produces a key `"summary"`, and `question_chain` knows to consume it.

---

2. **Multi-Output Support**
   - **`SequentialChain`** supports multiple outputs (`output_variables=["summary", "questions"]`).
   - This means you can explicitly request multiple outputs at the end of the chain and use them independently.
   - With `RunnableSequence`, it’s less intuitive to define multiple outputs, as it’s primarily designed for linear workflows where the final step provides a single result.

   **Example**:
   ```python
   sequential_chain = SequentialChain(
       chains=[summary_chain, question_chain],
       input_variables=["input"],
       output_variables=["summary", "questions"]
   )
   ```

---

3. **Reusability**
   - **`LLMChain`** provides a modular design:
     - Each chain (`summary_chain`, `question_chain`) is reusable and can be tested independently.
     - For instance, you could use the `summary_chain` in a different workflow without modifying the overall chain structure.
   - **`RunnableSequence`**, while modular for simpler workflows, does not explicitly separate and name each step, making reusability less straightforward.

   **Why Reusability Matters**:
   - If you wanted to replace the `question_chain` with a translation step (e.g., translating the summary to another language), you could simply swap out that chain without affecting the rest of the `SequentialChain`.



## Example: Sequential Chain - Paraphrasing and Sentiment Analysis

In [ ]:
# Step 1: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini", temperature=0.5)

# Step 2: Define the first prompt (paraphrasing)
# paraphrase_prompt = PromptTemplate(
#     input_variables=["text"],
#     template="Rewrite the following text to be more concise: {text}"
# )

# Clearer Instruction: Adding "much shorter" emphasizes that the rewrite should minimize wordiness.
paraphrase_prompt = PromptTemplate(
    input_variables=["text"],
    template="Rewrite the following text to be much shorter and concise while keeping the key points: {text}"
)

paraphrase_chain = LLMChain(prompt=paraphrase_prompt, llm=llm, output_key="paraphrased_text")

# Step 3: Define the second prompt (sentiment analysis)
sentiment_prompt = PromptTemplate(
    input_variables=["paraphrased_text"],
    template="Analyze the sentiment of the following text and classify it as positive, negative, or neutral: {paraphrased_text}"
)
sentiment_chain = LLMChain(prompt=sentiment_prompt, llm=llm, output_key="sentiment")

# Step 4: Create a sequential chain
sequential_chain = SequentialChain(
    chains=[paraphrase_chain, sentiment_chain],
    input_variables=["text"],  # Input to the first chain
    output_variables=["paraphrased_text", "sentiment"]  # Final output keys
)

# Step 5: Run the chain
input_text = (
    "I just watched the new movie 'Wicked' and I have to say, it completely exceeded my expectations! "
    "The performances were absolutely magical, with Elphaba's journey brought to life in such a captivating way. "
    "The visuals were stunning, from the Emerald City to the flying scenes, and the music gave me chills—it's everything a fan could hope for. "
    "Sure, there were a few small pacing issues, but overall, this was a breathtaking adaptation that I can't stop thinking about. "
    "I left the theater feeling inspired, uplifted, and ready to see it again!"
)

response = sequential_chain.invoke({"text": input_text})

# Step 6: Print the outputs
print("Original Text:", input_text)
print("\nParaphrased Text:", response["paraphrased_text"])
print("\nSentiment Analysis:", response["sentiment"])



Original Text: I just watched the new movie 'Wicked' and I have to say, it completely exceeded my expectations! The performances were absolutely magical, with Elphaba's journey brought to life in such a captivating way. The visuals were stunning, from the Emerald City to the flying scenes, and the music gave me chills—it's everything a fan could hope for. Sure, there were a few small pacing issues, but overall, this was a breathtaking adaptation that I can't stop thinking about. I left the theater feeling inspired, uplifted, and ready to see it again!

Paraphrased Text: I just watched 'Wicked' and it exceeded my expectations! The performances were magical, especially Elphaba's journey. The stunning visuals and chilling music made it a must-see for fans. Despite minor pacing issues, it was a breathtaking adaptation that left me inspired and eager to see it again!

Sentiment Analysis: The sentiment of the text is positive. The author expresses excitement and satisfaction with their exper

## Example: Sequential Chain - Summarization and Keyword Extraction

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain, LLMChain


# Step 1: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini", temperature=0.5)

# Step 2: Define the first prompt (Summarization)
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text into 2-3 sentences: {text}"
)
summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="summary")

# Step 3: Define the second prompt (Keyword Extraction)
keywords_prompt = PromptTemplate(
    input_variables=["summary"],
    template="From the following summary, extract the 5 most important keywords or topics:\n{summary}"
)
keywords_chain = LLMChain(prompt=keywords_prompt, llm=llm, output_key="keywords")

# Step 4: Create a sequential chain
sequential_chain = SequentialChain(
    chains=[summary_chain, keywords_chain],
    input_variables=["text"],  # Input to the first chain
    output_variables=["summary", "keywords"]  # Final output keys
)

# Step 5: Run the chain
input_text = (
    "The new movie 'Wicked' has captivated audiences with its stunning visuals, powerful performances, and iconic music. "
    "Elphaba's journey from an outcast to a powerful figure is portrayed beautifully, and the Emerald City scenes are breathtaking. "
    "Although some critics noted minor pacing issues, the overall production delivers an emotional and uplifting experience. "
    "Fans of the Broadway show will appreciate the faithful adaptation, while newcomers will be enchanted by the storytelling and visuals."
)

response = sequential_chain.invoke({"text": input_text})

# Step 6: Print the outputs
print("Original Text:", input_text)
print("\nSummary:", response["summary"])
print("\nKeywords:", response["keywords"])


Original Text: The new movie 'Wicked' has captivated audiences with its stunning visuals, powerful performances, and iconic music. Elphaba's journey from an outcast to a powerful figure is portrayed beautifully, and the Emerald City scenes are breathtaking. Although some critics noted minor pacing issues, the overall production delivers an emotional and uplifting experience. Fans of the Broadway show will appreciate the faithful adaptation, while newcomers will be enchanted by the storytelling and visuals.

Summary: The new movie 'Wicked' has impressed audiences with its stunning visuals, strong performances, and memorable music, effectively showcasing Elphaba's transformation from an outcast to a powerful figure. While some critics pointed out minor pacing issues, the overall production offers an emotional and uplifting experience that appeals to both fans of the Broadway show and newcomers alike.

Keywords: 1. Wicked
2. Elphaba
3. Visuals
4. Performances
5. Music




## Example: Three-Step Sequential Chain

#### Workflow Steps:
1. **Summarization**: Shorten the input text into a concise summary.
2. **Keyword Extraction**: Extract the 5 most important keywords or topics from the summary.
3. **Question Generation**: Generate 3 follow-up questions based on the extracted keywords.



In [ ]:
# Step 1: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini", temperature=0.5)

# Step 2: Define the first prompt (Summarization)
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text into 2-3 sentences: {text}"
)
summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="summary")

# Step 3: Define the second prompt (Keyword Extraction)
keywords_prompt = PromptTemplate(
    input_variables=["summary"],
    template="From the following summary, extract the 5 most important keywords or topics:\n{summary}"
)
keywords_chain = LLMChain(prompt=keywords_prompt, llm=llm, output_key="keywords")

# Step 4: Define the third prompt (Follow-Up Questions)
questions_prompt = PromptTemplate(
    input_variables=["keywords"],
    template="Using the following keywords, generate 3 engaging follow-up questions:\n{keywords}"
)
questions_chain = LLMChain(prompt=questions_prompt, llm=llm, output_key="follow_up_questions")

# Step 5: Create a sequential chain
sequential_chain = SequentialChain(
    chains=[summary_chain, keywords_chain, questions_chain],
    input_variables=["text"],  # Input to the first chain
    output_variables=["summary", "keywords", "follow_up_questions"]  # Final output keys
)

# Step 6: Run the chain
input_text = (
    "The new movie 'Wicked' has captivated audiences with its stunning visuals, powerful performances, and iconic music. "
    "Elphaba's journey from an outcast to a powerful figure is portrayed beautifully, and the Emerald City scenes are breathtaking. "
    "Although some critics noted minor pacing issues, the overall production delivers an emotional and uplifting experience. "
    "Fans of the Broadway show will appreciate the faithful adaptation, while newcomers will be enchanted by the storytelling and visuals."
)

response = sequential_chain.invoke({"text": input_text})

# Step 7: Print the outputs
print("Original Text:\n", input_text)
print("\nSummary:\n", response["summary"])
print("\nKeywords:\n", response["keywords"])
print("\nFollow-Up Questions:\n", response["follow_up_questions"])


Original Text:
 The new movie 'Wicked' has captivated audiences with its stunning visuals, powerful performances, and iconic music. Elphaba's journey from an outcast to a powerful figure is portrayed beautifully, and the Emerald City scenes are breathtaking. Although some critics noted minor pacing issues, the overall production delivers an emotional and uplifting experience. Fans of the Broadway show will appreciate the faithful adaptation, while newcomers will be enchanted by the storytelling and visuals.

Summary:
 The new movie 'Wicked' has impressed audiences with its stunning visuals, strong performances, and memorable music, effectively portraying Elphaba's transformation from an outcast to a powerful figure. While some critics pointed out minor pacing issues, the overall production offers an emotional and uplifting experience that appeals to both fans of the Broadway show and newcomers alike.

Keywords:
 1. Wicked
2. Elphaba
3. Visuals
4. Performances
5. Music

Follow-Up Questi

## Example: SEO Optimization

In [ ]:
# Step 1: Initialize the LLM
llm = ChatOpenAI(openai_api_key=api_key, model="gpt-4o-mini", temperature=0.5)

# Step 2: Define the first prompt (Summarization for Meta Description)
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following content into 2-3 sentences to be used as a meta description for SEO:\n{text}"
)
summary_chain = LLMChain(prompt=summary_prompt, llm=llm, output_key="meta_description")

# Step 3: Define the second prompt (Keyword Extraction)
keywords_prompt = PromptTemplate(
    input_variables=["meta_description"],
    template="From the following meta description, extract the 5 most relevant SEO keywords:\n{meta_description}"
)
keywords_chain = LLMChain(prompt=keywords_prompt, llm=llm, output_key="seo_keywords")

# Step 4: Define the third prompt (Generate Related Search Questions)
questions_prompt = PromptTemplate(
    input_variables=["seo_keywords"],
    template="Using the following keywords, generate 5 engaging search queries or follow-up questions that users might search for:\n{seo_keywords}"
)
questions_chain = LLMChain(prompt=questions_prompt, llm=llm, output_key="related_questions")

# Step 5: Create a sequential chain
seo_chain = SequentialChain(
    chains=[summary_chain, keywords_chain, questions_chain],
    input_variables=["text"],  # Input to the first chain
    output_variables=["meta_description", "seo_keywords", "related_questions"]  # Final outputs
)

# Step 6: Run the chain
input_text = (
    "The new movie 'Wicked' has captivated audiences with its stunning visuals, powerful performances, and iconic music. "
    "Elphaba's journey from an outcast to a powerful figure is portrayed beautifully, and the Emerald City scenes are breathtaking. "
    "Although some critics noted minor pacing issues, the overall production delivers an emotional and uplifting experience. "
    "Fans of the Broadway show will appreciate the faithful adaptation, while newcomers will be enchanted by the storytelling and visuals."
)

response = seo_chain.invoke({"text": input_text})

# Step 7: Print the outputs
print("Meta Description (SEO):\n", response["meta_description"])
print("\nSEO Keywords:\n", response["seo_keywords"])
print("\nRelated Search Queries:\n", response["related_questions"])


Meta Description (SEO):
 Experience the magic of 'Wicked,' a visually stunning film that brings Elphaba's transformative journey to life with powerful performances and iconic music. While some critics mention minor pacing issues, the adaptation captivates both Broadway fans and newcomers alike, delivering an emotional and uplifting cinematic experience. Don't miss this enchanting tale set against the breathtaking backdrop of the Emerald City.

SEO Keywords:
 1. Wicked
2. Elphaba
3. Broadway
4. cinematic experience
5. Emerald City

Related Search Queries:
 Sure! Here are five engaging search queries or follow-up questions based on the provided keywords:

1. "How does Elphaba's journey in Wicked compare to the original Wizard of Oz in the cinematic experience?"
2. "What are the must-see moments from Wicked on Broadway that bring the Emerald City to life?"
3. "Is there a film adaptation of Wicked featuring Elphaba, and when can we expect it to be released?"
4. "What makes the Broadway pro